In [1]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing
import os
import pandas as pd

In [2]:
r""" Build a word sequence-to-sequence training set """

from tensorflow.keras import preprocessing , utils

questions = list()
answers = list()

with open("movies-sequence-input.txt", "r") as file_input:
    movie_input = file_input.read()
df_input = pd.DataFrame(movie_input.split('\n'),columns=list('i'))
df_input = df_input.fillna(' ')

with open("movies-sequence-output.txt", "r") as file_output:
    movie_output = file_output.read()
df_output = pd.DataFrame(movie_output.split('\n'),columns=list('o'))
df_output = df_output.fillna(' ')

for input_text, target_text in zip(df_input.i, df_output.o):
    if(len(input_text)>400):
        questions.append(input_text[:400])
    else:
        questions.append(input_text)
    if(len(target_text)>400):
        answers.append(target_text[:400])
    else:
        answers.append(target_text)
answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))


VOCAB SIZE : 6709


In [3]:
# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

# Saving all the arrays to storage
np.save( 'enc_in_data.npy' , encoder_input_data )
np.save( 'dec_in_data.npy' , decoder_input_data )
np.save( 'dec_tar_data.npy' , decoder_output_data )


(5403, 81) 81
(5403, 89) 89
(5403, 89, 6709)


In [5]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()


Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 200)    1341800     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 200)    1341800     input_2[0][0]                    
_____________________________________

In [7]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=100 ) 
model.save( 'model3.h5' ) 

Epoch 1/100
5403/5403 [==============================] - 148s 27ms/sample - loss: 5.7940
Epoch 2/100
5403/5403 [==============================] - 147s 27ms/sample - loss: 5.3624
Epoch 3/100
5403/5403 [==============================] - 146s 27ms/sample - loss: 5.1783
Epoch 4/100
5403/5403 [==============================] - 147s 27ms/sample - loss: 5.0505
Epoch 5/100
5403/5403 [==============================] - 143s 26ms/sample - loss: 4.9370
Epoch 6/100
5403/5403 [==============================] - 146s 27ms/sample - loss: 4.8316
Epoch 7/100
5403/5403 [==============================] - 150s 28ms/sample - loss: 4.7356
Epoch 8/100
5403/5403 [==============================] - 146s 27ms/sample - loss: 4.6466
Epoch 9/100
5403/5403 [==============================] - 146s 27ms/sample - loss: 4.5642
Epoch 10/100
5403/5403 [==============================] - 148s 27ms/sample - loss: 4.4883
Epoch 11/100
5403/5403 [==============================] - 147s 27ms/sample - loss: 4.4156
Epoch 12/100
5403/5

5403/5403 [==============================] - 152s 28ms/sample - loss: 0.5040
Epoch 93/100
5403/5403 [==============================] - 148s 27ms/sample - loss: 0.4867
Epoch 94/100
5403/5403 [==============================] - 146s 27ms/sample - loss: 0.4672
Epoch 95/100
5403/5403 [==============================] - 144s 27ms/sample - loss: 0.4510
Epoch 96/100
5403/5403 [==============================] - 150s 28ms/sample - loss: 0.4346
Epoch 97/100
5403/5403 [==============================] - 151s 28ms/sample - loss: 0.4183
Epoch 98/100
5403/5403 [==============================] - 149s 28ms/sample - loss: 0.4059
Epoch 99/100
5403/5403 [==============================] - 146s 27ms/sample - loss: 0.3907
Epoch 100/100
5403/5403 [==============================] - 148s 27ms/sample - loss: 0.3748


In [8]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model


In [9]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')


In [16]:

enc_model , dec_model = make_inference_models()


for _ in range(10):
    prompt = input( 'Movie bot prompt : ' )
    if(prompt == 'end'):
        break
    states_values = enc_model.predict( str_to_tokens( prompt) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
                       
        if sampled_word == "end" or prompt == 'end' or len(decoded_translation.split()) > maxlen_answers:
            print("End")
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( "Movie_bot: " + decoded_translation )


Movie bot prompt : Hi how are you
End
Movie_bot:  oh no end
Movie bot prompt : what do you mean
End
Movie_bot:  i mean i'm a fucking fraud i'm no architect don't be a putz who's been to santiago twice in a year estadio olimpico please end
Movie bot prompt : is that really what you mean
End
Movie_bot:  of course i mean i mean i was now i was the one end
Movie bot prompt : excuse me
End
Movie_bot:  yes end
Movie bot prompt : that does not make sense
End
Movie_bot:  you do have how about me okay end
Movie bot prompt : this is fun
End
Movie_bot:  yeah i'm thinking of it didn't you wanted to look the girl who will be of an audition five but don't you know you're not going to run away from making the heart and end
Movie bot prompt : ran out of time
End
Movie_bot:  a little would be so be going to be having a willing with a loaded deal are you decided to go in the priest end
Movie bot prompt : what
End
Movie_bot:  you heard me you were right now end
Movie bot prompt : I guess this is good
End

In [ ]:
respond('Hi Rosa, how are you?')
respond('Hi Jim, how are you?')
respond('Hi Barak, how are you?')
respond('Hi Amy, how are you?')
respond('Hi Paris, how are you?')
respond('Hi Joe, how are you?')
respond('Hi Jane, how are you?')
respond('Hey Jane, how are you?')
respond('Hey Jon, how are you?')
respond('Hey John, how are you?')
respond('Hey Joe, how are you?')
respond('Hey Jim, how are you?')
respond('Hey Ashley, how are you?')
respond('Hey my love, how are you?')
respond('Hey Arzu, how are you?')
respond("I'm talking about us.")
respond("What are you trying to say?")
